In [0]:
baseDir = '/content/drive/My Drive/Exp8d'
!mkdir '/content/drive/My Drive/Exp8d'

mkdir: cannot create directory ‘/content/drive/My Drive/Exp8d’: File exists


In [0]:
import torch
from torchvision import transforms
from incClassifier import incClassifier
from torchvision import datasets
import torch.utils.data as data

In [0]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def visualize_model(model, dataloader, num_images=6):
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            if mymodel.useCuda:
              inputs = inputs.cuda()
              # labels = labels.cuda()

            preds, tasks, classes =  mymodel.predict(inputs)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(classes[j]))
                # ax.set_title('predicted: {}'.format(preds[j]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    # model.train(mode=was_training)
                    return

In [0]:
mymodel = incClassifier(True)
# mymodel.mainNet.cuda()

In [0]:
normal_transforms = {
    'train': transforms.Compose([
        transforms.Resize(35),
        transforms.RandomCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'val': transforms.Compose([
        transforms.Resize(35),
        transforms.CenterCrop(32),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
}

gray_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize(35),
        transforms.RandomCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.1307, 0.1307, 0.1307), (0.1307, 0.1307, 0.1307))
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize(35),
        transforms.CenterCrop(32),
        transforms.ToTensor(),
        transforms.Normalize((0.1307, 0.1307, 0.1307), (0.1307, 0.1307, 0.1307))
    ]),
}

scenes_path ='/home/ahmad/DataSet/scenes'
batch_size = 100


## MNIST FAssion

In [0]:
fmnist = {x: datasets.FashionMNIST(root='/FashionMNIST', train=True if x == 'train' else False,
                                        download=True, transform=gray_transforms[x])
         for x in ['train', 'val']}
fmnist_loaders = {x: torch.utils.data.DataLoader(fmnist[x], batch_size=batch_size,
                                               shuffle=True, num_workers=4)
                for x in ['train', 'val']}
fmnist_izes = {x: len(fmnist[x]) for x in ['train', 'val']}
fmnist_classes = ['T-Shirt','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot']

0it [00:00, ?it/s]

26427392it [00:04, 6044173.89it/s]                              


Extracting /FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /FashionMNIST/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 39939.89it/s]                           
0it [00:00, ?it/s]

Extracting /FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /FashionMNIST/FashionMNIST/raw


4423680it [00:02, 1667642.19it/s]                             
0it [00:00, ?it/s]

Extracting /FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /FashionMNIST/FashionMNIST/raw


8192it [00:00, 15405.02it/s]            

Extracting /FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /FashionMNIST/FashionMNIST/raw
Processing...
Done!


In [0]:
!mkdir '/content/drive/My Drive/Exp8d/checkpointFashionMNIST'
check_dir =  '/content/drive/My Drive/Exp8d/checkpointFashionMNIST'
mymodel.trainNewTask(fmnist, fmnist_loaders, fmnist_classes, check_dir)

In [0]:
torch.save(mymodel, '/content/drive/My Drive/Exp8d/mymodel.pt')
mymodel = torch.load('/content/drive/My Drive/Exp8d/mymodel.pt')

## SVHN

In [0]:
svhn = {x: datasets.SVHN(root='/SVHN', split=x if x == 'train' else 'test',
                                        download=True, transform=gray_transforms[x])
         for x in ['train', 'val']}
svhn_loaders = {x: torch.utils.data.DataLoader(svhn[x], batch_size=batch_size,
                                               shuffle=True, num_workers=4)
                for x in ['train', 'val']}
dset_sizes = {x: len(svhn[x]) for x in ['train', 'val']}
svhn_classes = [str(x) for x in range(10)]

0it [00:00, ?it/s]

 99%|█████████▉| 180895744/182040794 [00:11<00:00, 21001645.25it/s]
0it [00:00, ?it/s]


 95%|█████████▌| 61235200/64275384 [00:06<00:00, 21833329.99it/s]
64282624it [00:06, 10229691.81it/s]                              

In [0]:
!mkdir '/content/drive/My Drive/Exp8d/checkpointSVHN'
check_dir =  '/content/drive/My Drive/Exp8d/checkpointSVHN'
mymodel.trainNewTask(svhn, svhn_loaders, svhn_classes, check_dir)

In [0]:
torch.save(mymodel, '/content/drive/My Drive/Exp8d/mymodel.pt')
mymodel = torch.load('/content/drive/My Drive/Exp8d/mymodel.pt')

## MNIST

In [0]:
mnist = {x: datasets.MNIST(root='/MNIST', train=True if x == 'train' else False,
                                        download=True, transform=gray_transforms[x])
         for x in ['train', 'val']}
mnist_loaders = {x: torch.utils.data.DataLoader(mnist[x], batch_size=batch_size,
                                               shuffle=True, num_workers=4)
                for x in ['train', 'val']}
mnist_sizes = {x: len(mnist[x]) for x in ['train', 'val']}
mnist_classes = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']


0it [00:00, ?it/s]


100%|█████████▉| 9887744/9912422 [00:02<00:00, 1342723.43it/s]
9920512it [00:02, 4010328.36it/s]                             

Extracting /MNIST/MNIST/raw/train-images-idx3-ubyte.gz to /MNIST/MNIST/raw



0it [00:00, ?it/s]


 57%|█████▋    | 16384/28881 [00:00<00:00, 85745.54it/s]
32768it [00:00, 56739.29it/s]                           
0it [00:00, ?it/s]

Extracting /MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to /MNIST/MNIST/raw



 97%|█████████▋| 1597440/1648877 [00:01<00:00, 479696.35it/s]
1654784it [00:01, 884739.38it/s]                             
0it [00:00, ?it/s]

Extracting /MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to /MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]
8192it [00:00, 21735.02it/s]            

Extracting /MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to /MNIST/MNIST/raw
Processing...
Done!


In [0]:
!mkdir '/content/drive/My Drive/Exp8d/checkpointMNIST'
check_dir =  '/content/drive/My Drive/Exp8d/checkpointMNIST'
mymodel.trainNewTask(mnist, mnist_loaders, mnist_classes, check_dir)

In [0]:
torch.save(mymodel, '/content/drive/My Drive/Exp8d/mymodel.pt')
mymodel = torch.load('/content/drive/My Drive/Exp8d/mymodel.pt')

## CIFAR

In [0]:
cifar = {x: datasets.CIFAR10(root='/CIFAR10', train=True if x == 'train' else False,
                                        download=True, transform=normal_transforms[x])
         for x in ['train', 'val']}
cifar_loaders = {x: torch.utils.data.DataLoader(cifar[x], batch_size=batch_size,
                                               shuffle=True, num_workers=4)
                for x in ['train', 'val']}
cifar_sizes = {x: len(cifar[x]) for x in ['train', 'val']}
cifar_classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


0it [00:00, ?it/s]


  9%|▉         | 15900672/170498071 [00:03<01:37, 1588744.48it/s]
182042624it [00:30, 21001645.25it/s]                               
 99%|█████████▉| 168845312/170498071 [00:12<00:00, 16405583.30it/s]

Extracting /CIFAR10/cifar-10-python.tar.gz to /CIFAR10
Files already downloaded and verified


In [0]:
!mkdir '/content/drive/My Drive/Exp8d/cifar_checkpoint'
check_dir =  '/content/drive/My Drive/Exp8d/cifar_checkpoint'
mymodel.trainNewTask(cifar, cifar_loaders, cifar_classes, check_dir)

In [0]:
torch.save(mymodel, '/content/drive/My Drive/Exp8d/mymodel.pt')
mymodel = torch.load('/content/drive/My Drive/Exp8d/mymodel.pt')


170500096it [00:25, 16405583.30it/s]                               

##Testing

In [0]:
ntask = 2
test_corrects = 0
for i, (inputs, labels) in enumerate(fmnist_loaders['val']):
  inputs = inputs.cuda()
  labels = labels.cuda()
  preds, tasks, _ =  mymodel.predict(inputs)
  test_corrects += torch.sum((preds == labels.data))
fin_acc = test_corrects.double() / len(fmnist_loaders['val'].dataset)
print('Loss: -- Acc: {:.4f}'.format(fin_acc))

Loss: -- Acc: 0.1876


In [0]:
ntask = 0
test_corrects = 0
for i, (inputs, labels) in enumerate(svhn_loaders['val']):
  inputs = inputs.cuda()
  labels = labels.cuda()
  preds, tasks, _ =  mymodel.predict(inputs)
  test_corrects += torch.sum((preds == labels.data))
fin_acc = test_corrects.double() / len(svhn_loaders['val'].dataset)
print('Loss: -- Acc: {:.4f}'.format(fin_acc))

Loss: -- Acc: 0.2468


In [0]:
ntask = 1
test_corrects = 0
for i, (inputs, labels) in enumerate(mnist_loaders['val']):
  inputs = inputs.cuda()
  labels = labels.cuda()
  preds, tasks, _ =  mymodel.predict(inputs)
  test_corrects += torch.sum((preds == labels.data))
fin_acc = test_corrects.double() / len(mnist_loaders['val'].dataset)
print('Loss: -- Acc: {:.4f}'.format(fin_acc))

Loss: -- Acc: 0.4687


In [0]:
ntask = 0
test_corrects = 0
for i, (inputs, labels) in enumerate(cifar_loaders['val']):
  inputs = inputs.cuda()
  labels = labels.cuda()
  preds, tasks, _ =  mymodel.predict(inputs)
  test_corrects += torch.sum((preds == labels.data))
fin_acc = test_corrects.double() / len(cifar_loaders['val'].dataset)
print('Loss: -- Acc: {:.4f}'.format(fin_acc))

Loss: -- Acc: 0.5751


In [0]:
# visualize_model(mymodel, dataloader, num_images=6)